# Functions evaluation

In [3]:
import json
import urllib.request as ur
from tqdm.notebook import tqdm
import datetime as dt

import xarray as xr
import numpy as np
import pandas as pd

import lmfit
import numdifftools
import corner
from scipy.stats import f_oneway

from matplotlib import pyplot as plt

from IPython.display import display, Markdown

_np = np.seterr(all="ignore", divide="raise")


Bad key "text.kerning_factor" on line 4 in
/Users/massimopierini/opt/anaconda3/envs/covid/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
url = 'https://raw.githubusercontent.com/maxdevblock/covid-19-time-series/master/json/COVID-COUNTRIES.json'
with ur.urlopen(url) as _url:
    _data = json.loads(_url.read().decode())

In [3]:
def gompertz_function(x, a, b, k, e):
    exp = - np.exp(k * (b - x))
    return a * np.exp(exp) + e

In [4]:
def double_gompertz_function(x, a1, b1, k1, a2, b2, k2, e):
    exp1 = - np.exp(k1 * (b1 - x))
    g1 = a1 * np.exp(exp1)
    exp2 = - np.exp(k2 * (b2 - x))
    g2 = (a2 - a1) * np.exp(exp2)
    return g1 + g2 + e

In [5]:
def logit_function(x, a, b, k, e):
    d = k * (b - np.array(x))
    return (a / (1 + np.exp(d))) + e

In [6]:
def double_logit_function(x, a1, b1, k1, a2, b2, k2, e):
    d1 = k1 * (b1 - np.array(x))
    l1 = a1 / (1 + np.exp(d1))
    d2 = k2 * (b2 - np.array(x))
    l2 = (a2 - a1) / (1 + np.exp(d2))
    return l1 + l2 + e

In [7]:
def guessed_params(function, model, y):
    dy = pd.Series(y).diff()
    max_y_i = y.argmax()
    
    if function == gompertz_function:
        p = model.make_params(
            a=y[-1],
            b=max_y_i,
            k=.1,
            e=y[0]
        )
    
    if function == logit_function:
        p = model.make_params(
            a=y[-1],
            b=max_y_i,
            k=.1,
            e=y[0]
        )
    
    if function in [double_logit_function, double_gompertz_function]:
        p = model.make_params(
            a1=y[max_y_i] * 2,
            b1=max_y_i,
            k1=.1,
            a2=max(y),
            b2=len(y),
            k2=.1,
            e=y[0]
        )
    
    return p

In [8]:
def get_stats(country, function, print_res=False):
    x_dates = np.array(list(_data[country]["Confirmed"].keys()))
    x = np.arange(0, len(x_dates))
    y = np.array(list(_data[country]["Confirmed"].values()))
    
    # define model
    model = lmfit.Model(function)
    
    p = guessed_params(function, model, y)
    
    # get model fit optimal results
    result = model.fit(data=y, params=p, x=x, method='Nelder', nan_policy='omit')

    if print_res:
        lmfit.report_fit(result)
        result.plot()
        plt.show()
    
        # get stats for model fit
        emcee_kws = dict(
            steps=1000, burn=300, thin=20, is_weighted=False, progress=False
        )
        emcee_params = result.params.copy()
        emcee_params.add('__lnsigma', value=np.log(0.1), min=np.log(0.001), max=np.log(2.0))

        result_emcee = model.fit(
            data=y, x=x, params=emcee_params, method='emcee',
            nan_policy='omit', fit_kws=emcee_kws
        )

        lmfit.report_fit(result_emcee)

        ax = plt.plot(x, model.eval(params=result.params, x=x), label='Nelder', zorder=100)
        result_emcee.plot_fit(ax=ax, data_kws=dict(color='gray', markersize=2))
        plt.show()

        plt.plot(result_emcee.acceptance_fraction)
        plt.xlabel('walker')
        plt.ylabel('acceptance fraction')
        plt.show()

        if hasattr(result_emcee, "acor"):
            print("Autocorrelation time for the parameters:")
            print("----------------------------------------")
            for i, p in enumerate(result.params):
                print(p, result.acor[i])

        emcee_corner = corner.corner(
            result_emcee.flatchain, labels=result_emcee.var_names,
            truths=list(result_emcee.params.valuesdict().values())
        )
        plt.show()

    return result

In [10]:
functions = [gompertz_function, logit_function, double_gompertz_function, double_logit_function]

cols = ["countries"]
for function in functions:
    func_name = function.__name__.replace("_", " ").title()
    cols.append(func_name)

results = []

for i, country in enumerate(tqdm(list(_data.keys()))):
    results.append([f"{country}"])
    results[i].append(np.array(list(_data[country]["Confirmed"].values())))
    f_stats = {"best_fit": [], "residual": []}
    for function in functions:
        try:
            result = get_stats(country, function, print_res=False)
            results[i].append(result)
        except Exception as err:
            print(f"{country} - {function} ERR {err}")

Yemen - <function gompertz_function at 0x1a1a4c8a70> ERR divide by zero encountered in log
Yemen - <function logit_function at 0x1a1a4c8e60> ERR divide by zero encountered in double_scalars



In [11]:
cols = ["countries", "measured"]
for function in functions:
    func_name = function.__name__.replace("_", " ").title()
    cols.append(func_name)

res_df = pd.DataFrame(data=results, columns=cols)
res_df.set_index("countries", inplace=True)

In [18]:
today = dt.datetime.now().strftime("%Y-%m-%d")
res_df.to_pickle(f"data-confirmed-{today}.pkl")

'2020-04-29'

In [157]:
res_df["Logit Function"]["China"]

In [2]:
res_df["measured"]

NameError: name 'res_df' is not defined